In [ ]:
from abc import abstractmethod
from build.Decorators import debug

import plotly.graph_objects as go

In [1]:
class Player:
    """
    Parent class for all types of player
    """

    def __init__(self, representation_char) -> None:
        """
        :param representation_char: char to represent the player on the game board
        """
        self.representation_char = representation_char
        self.stats = Stats()

    def __str__(self) -> str:
        """
        __str__ method to output Player object with corresponding char
        :return: representation char
        """
        return str(self.representation_char)

    @abstractmethod
    @debug
    def make_move(self, board) -> int:
        """
        Every player has to make his own move
        :param board: current board state
        """
        pass

    @abstractmethod
    @debug
    def give_result(self, result) -> None:
        """
        Each player must receive and process feedback
        :param result: Enum element for feedback
        """
        pass

In [ ]:
def get_history_for_group(record, groups):

    won = []
    draw = []
    lost = []

    won_count = 0
    draw_count = 0
    lost_count = 0

    for group in groups:
        for i in range(group):
            if record[i] == 1:
                won_count += 1
            elif record[i] == 0:
                draw_count += 1
            elif record[i] == -1:
                lost_count += 1

        overall = won_count + draw_count + lost_count
        won.append(won_count / overall)
        draw.append(draw_count / overall)
        lost.append(lost_count / overall)

    return won, lost, draw

In [ ]:
class Stats:
    """
    An object tracking the stats of a player
    """

    def __init__(self):
        self._won = 0
        self._lost = 0
        self._draw = 0
        self._record = []
        self._history = []

    def display(self) -> None:

        games = self.played_games()

        print(f"Games     won    |    lost    |    draw    | total games")
        print(f"      {self.get_won():^10} | {self.get_lost():^10} | {self.get_draw():^10} | {games:^10}")
        print(f"      {self.get_won() / games:^10.0%} "
              f"| {self.get_lost() / games:^10.0%} "
              f"| {self.get_draw() / games:^10.0%} "
              f"| {games / games:^10.0%}")

    def incr_won(self) -> None:
        self._won += 1
        self._record.append(1)

    def incr_lost(self) -> None:
        self._lost += 1
        self._record.append(-1)

    def incr_draw(self) -> None:
        self._draw += 1
        self._record.append(0)

    def get_won(self) -> int:
        return self._won

    def get_lost(self) -> int:
        return self._lost

    def get_draw(self) -> int:
        return self._draw

    def played_games(self) -> int:
        return self.get_won() + self.get_lost() + self.get_draw()

    def get_record(self):
        return self._record

    def display_history(self, groups=None, group_count=5):

        if groups is None:
            record_size = len(self.get_record())
            step_size = record_size // group_count
            groups = list(range(step_size, record_size + 1, step_size))

        won, lost, draw = get_history_for_group(self.get_record(), groups)

        fig = go.Figure(data=[
            go.Bar(name='won', x=groups, y=won, marker={'color': '#a8ddb5'}),
            go.Bar(name='draw', x=groups, y=draw, marker={'color': '#9ebcda'}),
            go.Bar(name='lost', x=groups, y=lost, marker={'color': '#fc9272'})
        ])

        fig.update_layout(barmode='group', yaxis=dict(tickformat="%"))
        fig.show()